In [ ]:
# Detect brain tumors using the features collected from the data

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, Dropout, MaxPooling1D


path = '...'
path_out = path + 'Run/'

if not os.path.exists(path_out):
    os.makedirs(path_out)
    print("Directory Created")
else:    
    print("Directory Already Exists") 

img_dataset = np.genfromtxt(path + 'All_Features_only_values.csv', delimiter=',')

num_rows = len(img_dataset)
num_cols = len(img_dataset[0])


X = np.zeros(shape=(num_rows, 19))
y = np.zeros(num_rows)

# Randomly shuffle read data
random.shuffle(img_dataset)

for i in range(0, num_rows):
    X[i,0] = img_dataset[i,0]
    X[i,1] = img_dataset[i,1]
    X[i,2] = img_dataset[i,2]
    X[i,3] = img_dataset[i,3]
    X[i,4] = img_dataset[i,4]
    X[i,5] = img_dataset[i,5]
    X[i,6] = img_dataset[i,6]
    X[i,7] = img_dataset[i,7]
    X[i,8] = img_dataset[i,8]
    X[i,9] = img_dataset[i,9]
    X[i,10] = img_dataset[i,10]
    X[i,11] = img_dataset[i,11]
    X[i,12] = img_dataset[i,12]
    X[i,13] = img_dataset[i,13]
    X[i,14] = img_dataset[i,14]
    X[i,15] = img_dataset[i,15]
    X[i,16] = img_dataset[i,16]
    X[i,17] = img_dataset[i,17]
    X[i,18] = img_dataset[i,18]
    y[i] = img_dataset[i,19]

X = X.reshape(X.shape[0], X.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
num_train = len(y_train)
num_test = len(y_test)

X_train = np.array(X_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

X_train1, X_val, y_train1, y_val = train_test_split(X_train, y_train, test_size = 0.20)
X_train1 = np.array(X_train1).astype(np.float32)
y_train1 = np.array(y_train1).astype(np.float32)
X_val = np.array(X_val).astype(np.float32)
y_val = np.array(y_val).astype(np.float32)
num_train1 = len(y_train1)
num_val = len(y_val)

print(X_train1.shape[1])
print(X_val.shape)

file0 = open(path_out + "training_test.txt","w")#append mode 
file0.write("Number of total data : " + str(num_rows)) 
file0.write("\n")
file0.write("Number of training data : " + str(num_train)) 
file0.write("\n")
file0.write("Number of testing data : " + str(num_test)) 
file0.write("\n \n \n")
file0.write("Training data was divided into train and validation.")
file0.write("\n")
file0.write("Number of training data (final) : " + str(num_train1)) 
file0.write("\n")
file0.write("Number of validation data : " + str(num_val)) 
file0.close()
print("Training, Validation and Test Data Splitted")

model = Sequential()

# 1st Layer
model.add(Conv1D(64, 2, activation="relu", input_shape=(19,1)))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Dropout(0.2))
# Output Layer
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['acc'])

# Training
history = model.fit(X_train1, y_train1, 
                    epochs = 100, 
                    validation_data = (X_val, y_val))

# Save model
model.save_weights(path_out + 'model_weights.h5')
model.save(path_out + 'model_keras.h5')

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

avg_acc = np.mean(acc) * 100
avg_val_acc = np.mean(val_acc) * 100
avg_loss = np.mean(loss) * 100
avg_val_loss = np.mean(val_loss) * 100
file1 = open(path_out + "avg_eval.txt","w")#append mode 
file1.write("Average Training Accuracy: " + str(avg_acc)) 
file1.write("\n")
file1.write("Average Validation Accuracy: " + str(avg_val_acc)) 
file1.write("\n")
file1.write("Average Training Loss: " + str(avg_loss)) 
file1.write("\n")
file1.write("Average Validation Loss: " + str(avg_val_loss)) 
file1.close()

np.savetxt(path_out + "train_acc.txt", acc, delimiter="\n") 
np.savetxt(path_out + "val_acc.txt", val_acc, delimiter="\n") 
np.savetxt(path_out + "train_loss.txt", loss, delimiter="\n") 
np.savetxt(path_out + "val_loss.txt", val_loss, delimiter="\n") 

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.savefig(path_out + "Accuracy.jpg")
plt.show()

plt.plot(epochs, loss, 'b', label = 'Training Loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
plt.title('Training Loss')
plt.legend()
plt.savefig(path_out + "Loss.jpg")
plt.show()

np.savetxt(path_out + "y_test.txt", y_test, delimiter="\n") 

y_pred = []
p = 1

pred = model.predict(X_test)

test_eval = model.evaluate(X_test, y_test, verbose=0)
test_acc = test_eval[1] * 100
test_loss = test_eval[0] * 100
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

file2 = open(path_out + "test_eval.txt","w")#append mode 
file2.write("Test Accuracy: " + str(test_acc)) 
file2.write("\n")
file2.write("Test Loss: " + str(test_loss)) 
file2.close() 

np.savetxt(path_out + "y_prediction.txt", pred, delimiter="\n") 

a = 0

for value in pred:
    if (pred[a] > 0.5):
        y_pred.append(1)
        a = a + 1
    else:
        y_pred.append(0)
        a = a + 1

np.savetxt(path_out + "y_pred.txt", y_pred, delimiter="\n")

c = 0

for i in range(0, num_test):
    a = y_test[i]
    b = y_pred[i]
    if(a==b):
        c = c + 1
    
pred_acc = (c/num_test) * 100

file3 = open(path_out + "pred_acc.txt","w")#append mode 
file3.write("Correct Predictions: " + str(c)) 
file3.write("\n")
file3.write("Prediction Accuracy: " + str(pred_acc)) 
file3.close() 

print('Total images in Test set :' + str(num_test))
print('Correctly predicted label: ', c)

print('Prediction Accuracy : ', pred_acc)

num_classes = 2

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))
clsf_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, output_dict=True)).transpose()
clsf_report.to_csv(path_out + "Classification Report.csv", index= True)